In [1]:
import sqlite3
from tkinter import *
from tkinter import messagebox  # messagebox modülünü ekledik
from tkinter import simpledialog

class LibrarySystem:
    def __init__(self):
        super().__init__()
        self.conn = sqlite3.connect("library.db")
        self.cur = self.conn.cursor()
        self.cur.execute("CREATE TABLE IF NOT EXISTS books (id INTEGER PRIMARY KEY, title TEXT, author TEXT, status TEXT)")
        self.conn.commit()

    def add_book(self, title, author):
        self.cur.execute("INSERT INTO books VALUES (NULL,?,?,?)", (title, author, "kiralık"))
        self.conn.commit()

    def delete_book(self, id):
        self.cur.execute("DELETE FROM books WHERE id=?", (id,))
        self.conn.commit()

    def search_books(self, title):
        if title:
            # Küçük harfe dönüştürülen kitap adı ile karşılaştırma yapılıyor
            self.cur.execute("SELECT * FROM books WHERE LOWER(title)=?", (title.lower(),))
        else:
            self.cur.execute("SELECT * FROM books")
        books = self.cur.fetchall()
        return books

    def view_returned_books(self):
        self.cur.execute("SELECT * FROM books WHERE status=?", ("kiralık",))
        books = self.cur.fetchall()
        return books

    def borrow_book(self, id):
        self.cur.execute("UPDATE books SET status=? WHERE id=?", ("kiralandı", id))
        self.conn.commit()

    def return_book(self, id):
        self.cur.execute("UPDATE books SET status=? WHERE id=?", ("kiralık", id))
        self.conn.commit()

class LibraryApplication:
    def __init__(self, window):
        self.library_system = LibrarySystem()
        self.is_admin = False  # Admin giriş durumu

        self.window = window
        self.window.title("Kütüphane Otomasyonu")
        self.window.geometry("650x350")

        # Labels
        self.titleLabel = Label(text="Kitap Adı", font=("bold", 12))
        self.titleLabel.grid(column=0, row=0)

        self.authorLabel = Label(text="Yazar", font=("bold", 12))
        self.authorLabel.grid(column=0, row=1)

        self.adminPasswordLabel = Label(text="Admin Şifresi", font=("bold", 12))
        self.adminPasswordLabel.grid(column=0, row=2)

        # Entry fields
        self.titleEntry = Entry()
        self.titleEntry.grid(column=1, row=0)

        self.authorEntry = Entry()
        self.authorEntry.grid(column=1, row=1)

        self.adminPasswordEntry = Entry(show="*")
        self.adminPasswordEntry.grid(column=1, row=2)

        # Buttons
        self.addButton = Button(text="Kitap Ekle", command=self.add_book, bg="lightblue", height=2, width=20, state=DISABLED)
        self.deleteButton = Button(text="Kitap Sil", command=self.delete_book, bg="lightpink", height=2, width=20, state=DISABLED)
        self.searchButton = Button(text="Kitap Ara", command=self.search_books, bg="lightgreen", height=2, width=20)
        self.returnedButton = Button(text="Kiralık Kitapları Gör", command=self.view_returned_books, bg="lightyellow", height=2, width=20)
        self.borrowButton = Button(text="Kitap Ödünç Al", command=self.borrow_book_button_clicked, bg="lightpink", height=2, width=20)
        self.returnButton = Button(text="Kitap İade", command=self.return_book, bg="lightcoral", height=2, width=20)

        self.addButton.grid(column=0, row=3, padx=5, pady=5)
        self.deleteButton.grid(column=1, row=3, padx=5, pady=5)
        self.searchButton.grid(column=0, row=4, padx=5, pady=5)
        self.returnedButton.grid(column=1, row=4, padx=5, pady=5)
        self.borrowButton.grid(column=0, row=5, padx=5, pady=5)
        self.returnButton.grid(column=1, row=5, padx=5, pady=5)
        self.viewAllButton = Button(text="Tüm kitapları gör", command=self.view_all_books, bg="lightcyan", height=2, width=20)
        self.viewAllButton.grid(column=0, row=6, padx=5, pady=5)

        # Login button
        self.loginButton = Button(text="Giriş Yap", command=self.admin_login, bg="lightgray", height=2, width=20)
        self.loginButton.grid(column=1, row=6, padx=5, pady=5)

        # Book list
        self.bookList = Listbox(width=50)
        self.bookList.grid(column=2, row=2, rowspan=9, columnspan=5, padx=5, pady=5)

    def add_book(self):
        title = self.titleEntry.get()
        author = self.authorEntry.get()
        self.library_system.add_book(title, author)
        self.bookList.delete(0, END)  # Listbox'u temizle
        self.titleEntry.delete(0, END)
        self.authorEntry.delete(0, END)
  
    def delete_book(self):
        selected_book = self.bookList.get(ACTIVE)
        book_id = selected_book.split(".")[0]
        book_status = self.library_system.cur.execute("SELECT status FROM books WHERE id=?", (book_id,)).fetchone()[0]

        if book_status == "kiralandı":
            messagebox.showerror("Hata", "Bu kitap kiralandığı için silemezsiniz.")
        else:
            self.library_system.delete_book(book_id)
            self.refresh_book_list()
            self.refresh_book_ids()  # Kitap ID'lerini yeniden sırala

        self.bookList.delete(0, END)  # Listbox'u temizle
        self.titleEntry.delete(0, END)
        self.authorEntry.delete(0, END)

    def search_books(self):
        title = self.titleEntry.get()
        books = self.library_system.search_books(title)
        self.bookList.delete(0, END)
        for book in books:
            self.bookList.insert(END, f"{book[0]}. {book[1]} by {book[2]} ({book[3]})")
        self.titleEntry.delete(0, END)
        self.authorEntry.delete(0, END)

    def view_returned_books(self):
        books = self.library_system.view_returned_books()
        self.bookList.delete(0, END)
        for book in books:
            self.bookList.insert(END, f"{book[0]}. {book[1]} by {book[2]} ({book[3]})")

    def borrow_book(self):
        selected_book = self.bookList.get(ACTIVE)
        book_id = selected_book.split(".")[0]
        self.library_system.borrow_book(book_id)
        self.refresh_book_list()
        self.bookList.delete(0, END)  # Listbox'u temizle
        self.titleEntry.delete(0, END)
        self.authorEntry.delete(0, END)
        
    def borrow_book_button_clicked(self):
        if self.is_admin:
            self.borrow_book()
        else:
            password = simpledialog.askstring("Kurum Ortak Şifresi", "Kurum Ortak Şifresini Girin", show="*")
            if password and self.check_common_password(password):
                self.borrow_book()
            else:
                messagebox.showerror("Hata", "Geçersiz Kurum Ortak Şifresi!")
 
    def return_book(self):
        selected_book = self.bookList.get(ACTIVE)
        book_id = selected_book.split(".")[0]
        if self.is_admin:
            self.library_system.return_book(book_id)
            self.refresh_book_list()
        else:
            password = simpledialog.askstring("Kurum Ortak Şifresi", "Kurum Ortak Şifresini Girin", show="*")
            if password and self.check_common_password(password):
                self.library_system.return_book(book_id)
                self.refresh_book_list()
            else:
                messagebox.showerror("Hata", "Geçersiz Kurum Ortak Şifresi!")

        self.bookList.delete(0, END)  # Listbox'u temizle
        self.titleEntry.delete(0, END)
        self.authorEntry.delete(0, END)

    def view_all_books(self):
        books = self.library_system.search_books("")  # Tüm kitapları getir
        self.bookList.delete(0, END)
        for book in books:
            self.bookList.insert(END, f"{book[0]}. {book[1]} by {book[2]} ({book[3]})")
        self.refresh_book_ids()  # Kitap ID'lerini yeniden sırala
      
    def admin_login(self):
        password = self.adminPasswordEntry.get()
        if password == "admin123":  # Admin şifresi kontrolü
            self.is_admin = True
            self.loginButton.config(text="Çıkış Yap", command=self.admin_logout, bg="orange")
            self.deleteButton.config(state=NORMAL)
            self.addButton.config(state=NORMAL)
            self.borrowButton.config(state=DISABLED)
            self.returnButton.config(state=DISABLED)
            self.viewAllButton.config(state=NORMAL)  # Tüm kitapları gör düğmesini etkinleştir
        else:
            messagebox.showerror("Hata", "Geçersiz admin şifresi!")
        self.adminPasswordEntry.delete(0, END)

    def admin_logout(self):
        self.is_admin = False
        self.loginButton.config(text="Giriş Yap", command=self.admin_login, bg="lightgray")
        self.deleteButton.config(state=DISABLED)
        self.addButton.config(state=DISABLED)
        self.borrowButton.config(state=NORMAL)
        self.returnButton.config(state=NORMAL)
        self.bookList.delete(0, END)  # Listbox'u temizle
        self.titleEntry.delete(0, END)
        self.authorEntry.delete(0, END)
        
    def refresh_book_list(self):
        self.bookList.delete(0, END)  # Listbox'u temizle
        books = self.library_system.search_books("")
        for book in books:
            self.bookList.insert(END, f"{book[0]}. {book[1]} by {book[2]} ({book[3]})")
    
    def refresh_book_ids(self):
        books = self.library_system.search_books("")
        for index, book in enumerate(books, start=1):
            book_id = book[0]
            self.library_system.cur.execute("UPDATE books SET id=? WHERE id=?", (index, book_id))
            self.library_system.conn.commit()
            
    def perform_search(self):
        title = self.titleEntry.get()
        self.search_books(title)
        
    def check_common_password(self, password):
        return password == "555"

if __name__ == "__main__":
    window = Tk()
    app = LibraryApplication(window)
    window.mainloop()
